CAR DETECTION MODEL

In [1]:
import numpy as np
import cv2
import os

In [2]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from tkinter import Label
from PIL import Image, ImageTk

Function to load and display images

In [5]:
def display_images_from_directory(directory):
    files = os.listdir(directory)
    for file in files:
        file_path = os.path.join(directory, file)
        image = cv2.imread(file_path)
        if image is not None:
            cv2.imshow("Image", image)
            key = cv2.waitKey(0)
            if key == ord('q'):
                print("Quitting...")
                break
        else:
            print(f"Could not load image: {file_path}")

    cv2.destroyAllWindows()

In [13]:
display_images_from_directory("Traffic Images/training")

Quitting...


In [12]:
display_images_from_directory("Traffic Images/testing")

Quitting...


In [14]:
# Paths to training and testing directories
train_dir = "Traffic Images/training"
test_dir = "Traffic Images/testing"

load the Haar Cascade for car detection

In [16]:
car_cascade_path = 'cars.xml'  
car_cascade = cv2.CascadeClassifier(car_cascade_path)


Load the COCO class labels

In [17]:
weights_path = "yolov3.weights" 
config_path = "yolov3.cfg"
labels_path = "coco.names"   

In [18]:
with open(labels_path, 'r') as f:
    labels = f.read().strip().split("\n")
print(labels)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


Load YOLOv3 model

In [19]:
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]


Using the UTKFace dataset for Gender classification

In [22]:
UTK = "UTKFace"

In [23]:
data = []
labels = []

for img_name in os.listdir(UTK):
    img_path = os.path.join(UTK, img_name)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (64, 64))
    label = int(img_name.split("_")[1])  # Gender label
    data.append(img)
    labels.append(label)

data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [26]:
# Split the data

from sklearn.model_selection import train_test_split

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=42)

In [28]:
# Build the model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

C:\Users\SARATHLAL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
# Compile the model
model.compile(loss="binary_crossentropy", optimizer=Adam(0.001), metrics=["accuracy"])

In [30]:
# Train the model
model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=32, epochs=10, verbose=1)

Epoch 1/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 32s 49ms/step - accuracy: 0.7319 - loss: 0.5326 - val_accuracy: 0.8577 - val_loss: 0.3198
Epoch 2/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 29s 48ms/step - accuracy: 0.8569 - loss: 0.3196 - val_accuracy: 0.8804 - val_loss: 0.2804
Epoch 3/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 29s 48ms/step - accuracy: 0.8799 - loss: 0.2794 - val_accuracy: 0.8827 - val_loss: 0.2674
Epoch 4/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step - accuracy: 0.8929 - loss: 0.2500 - val_accuracy: 0.8855 - val_loss: 0.2540
Epoch 5/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step - accuracy: 0.9000 - loss: 0.2354 - val_accuracy: 0.8946 - val_loss: 0.2503
Epoch 6/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 29s 50ms/step - accuracy: 0.9077 - loss: 0.2111 - val_accuracy: 0.8914 - val_loss: 0.2569
Epoch 7/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 29s 50ms/step - accuracy: 0.9158 - loss: 0.2028 - val_accuracy: 0.8954 - val_loss: 0.2486
Epoch 8/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step - accuracy: 0.9192 - loss: 0.1886 - 

In [31]:
# Save the model
model.save('gender_classification_model.h5')

In [32]:
# Load the gender classification model
gender_model = load_model('gender_classification_model.h5')

Function to detect objects in an image

In [34]:
def detect_objects(image):
    height, width, channels = image.shape
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Filter out weak detections
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    filtered_boxes = [boxes[i] for i in range(len(boxes)) if i in indexes]
    filtered_classes = [class_ids[i] for i in range(len(class_ids)) if i in indexes]
    return filtered_boxes, filtered_classes


Function to detect cars using Haar Cascade

In [35]:
def detect_cars(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cars = car_cascade.detectMultiScale(gray, 1.1, 1)
    car_boxes = [[x, y, w, h] for (x, y, w, h) in cars]
    return car_boxes


Function to classify car colors

In [36]:
def classify_car_color(image, boxes):
    colors = []
    for box in boxes:
        x, y, w, h = box
        car_image = image[y:y+h, x:x+w]
        avg_color = np.mean(car_image, axis=(0, 1))
        if avg_color[2] > avg_color[0]:
            colors.append('blue')
        else:
            colors.append('red')
    return colors



Function to predict gender

In [37]:
def predict_gender(image, boxes):
    males, females = 0, 0
    for box in boxes:
        x, y, w, h = box
        person_image = image[y:y+h, x:x+w]
        person_image = cv2.resize(person_image, (64, 64)) / 255.0
        person_image = np.expand_dims(person_image, axis=0)
        gender = gender_model.predict(person_image)
        if gender[0][0] > 0.5:
            females += 1
        else:
            males += 1
    return males, females

Function to process the traffic image

In [38]:
def process_traffic_image(image):
    boxes, classes = detect_objects(image)
    car_boxes = detect_cars(image)
    person_boxes = [box for box, cls in zip(boxes, classes) if cls == 0]  # Class 0 for people

    # Swap car colors
    car_colors = classify_car_color(image, car_boxes)
    swapped_colors = ['red' if color == 'blue' else 'blue' for color in car_colors]

    # Predict gender for people
    males, females = predict_gender(image, person_boxes)

    # Count other vehicles
    other_vehicle_count = sum(1 for cls in classes if cls not in [0, 2])

    return swapped_colors, len(car_boxes), males, females, other_vehicle_count, car_boxes + person_boxes, ['car'] * len(car_boxes) + ['person'] * len(person_boxes)


Function to display the detection results on the image

In [39]:
def draw_detections(image, boxes, classes):
    for i, box in enumerate(boxes):
        x, y, w, h = box
        label = classes[i]
        color = (0, 255, 0)  # Green color for rectangles
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return image


Function to open and process a single image file

In [40]:
def open_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        image = cv2.imread(file_path)
        if image is not None:
            results = process_traffic_image(image)
            image_with_detections = draw_detections(image, results[5], results[6])

            # Convert the image to RGB and display in the GUI
            image_rgb = cv2.cvtColor(image_with_detections, cv2.COLOR_BGR2RGB)
            im_pil = Image.fromarray(image_rgb)
            imgtk = ImageTk.PhotoImage(image=im_pil)

            img_label.config(image=imgtk)
            img_label.image = imgtk

            result_text = f"Swapped Colors: {results[0]}\n" \
                          f"Number of Cars: {results[1]}\n" \
                          f"Number of Males: {results[2]}\n" \
                          f"Number of Females: {results[3]}\n" \
                          f"Other Vehicles: {results[4]}"
            result_label.config(text=result_text)

        else:
            messagebox.showerror("Error", f"Could not load image: {file_path}")


Setting up the GUI

In [41]:
root = tk.Tk()
root.title("Traffic Signal Analysis")

btn_open_image = tk.Button(root, text="Upload Image", command=open_image)
btn_open_image.pack(pady=20)

img_label = Label(root)
img_label.pack(pady=10)

result_label = Label(root, text="", justify=tk.LEFT)
result_label.pack(pady=10)

root.mainloop()